In [ ]:
# Umbrella AI
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import math    
import operator
#

In [2]:
# Dropping a graph from the schema

#print(conn.gsql('''USE GLOBAL DROP GRAPH Umbrella_AI''', options=[]))

# print(conn.gsql('''USE GLOBAL DROP ALL''', options=[]))

In [3]:
!pip list

Package                            Version
---------------------------------- --------------------
absl-py                            1.3.0
alabaster                          0.7.12
anaconda-client                    1.9.0
anaconda-navigator                 2.3.2
anaconda-project                   0.10.1
anyio                              2.2.0
appdirs                            1.4.4
applaunchservices                  0.2.1
appnope                            0.1.2
appscript                          1.1.2
argh                               0.26.2
argon2-cffi                        20.1.0
arrow                              0.13.1
asn1crypto                         1.4.0
astroid                            2.6.6
astropy                            4.3.1
astunparse                         1.6.3
async-generator                    1.10
atomicwrites                       1.4.0
attrs                              21.2.0
autopep8                           1.5.7
Babel                              

In [4]:
pip install pyTigerGraph

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install pyTigerDriver 

Note: you may need to restart the kernel to use updated packages.


In [6]:
!python -m pip uninstall pyTigerGraph --yes

Found existing installation: pyTigerGraph 1.2.6
Uninstalling pyTigerGraph-1.2.6:
  Successfully uninstalled pyTigerGraph-1.2.6


In [7]:
!python -m pip uninstall pyTigerDriver --yes

Found existing installation: pyTigerDriver 1.0.15
Uninstalling pyTigerDriver-1.0.15:
  Successfully uninstalled pyTigerDriver-1.0.15


In [8]:
# FETCH PACKAGES

!pip install pyTigerGraph

     |████████████████████████████████| 203 kB 1.2 MB/s eta 0:00:01
  Using cached pyTigerDriver-1.0.15-py3-none-any.whl (12 kB)


In [ ]:
import pyTigerGraph as tg
import json

In [ ]:
# Establish the initial connetion with Connection parameters

hostName = "https://86ed106bae1640d6b4937d55b6997e02.i.tgcloud.io"
graphName = "UmbrellaAI2"
secret = "idp021i4h3dbledu6me1rrvbv8ui21sl"

conn = tg.TigerGraphConnection(host=hostName, graphname=graphName, gsqlSecret=secret)

#print("Connected")

In [4]:
secret = "idp021i4h3dbledu6me1rrvbv8ui21sl"

#use the secret to get a token
authToken = conn.getToken(secret)[0]

#connect to graph with token
conn = tg.TigerGraphConnection(host=hostName, graphname=graphName, gsqlSecret=secret, apiToken=authToken)

/Users/mac/opt/anaconda3/lib/python3.9/site-packages/pyTigerGraph/pyTigerGraphBase.py:106: DeprecationWarning: The `apiToken` parameter is deprecated; use `getToken()` function instead.
  warnings.warn(


# DEFINE / CREATE ALL VERTICES

In [5]:
results = conn.gsql('''
    USE GLOBAL
     CREATE VERTEX Patients(PRIMARY_ID id STRING, first_name STRING, last_name STRING, address STRING, age STRING, email STRING, gender STRING, phone STRING) WITH primary_id_as_attribute = "true"
     CREATE VERTEX Symptoms(PRIMARY_ID id STRING, symptoms STRING) WITH primary_id_as_attribute = "true"
     CREATE VERTEX Disease(PRIMARY_ID id STRING, disease STRING) WITH primary_id_as_attribute = "true"
     CREATE VERTEX Specialist(PRIMARY_ID id STRING, specialist STRING) WITH primary_id_as_attribute = "true"
''')

print(results)

Successfully created vertex types: [Patients].
Successfully created vertex types: [Symptoms].
Successfully created vertex types: [Disease].
Successfully created vertex types: [Specialist].


# DEFINE / CREATE ALL EDGES

In [6]:
results = conn.gsql('''
    USE GLOBAL
    CREATE DIRECTED EDGE patient_shows_on_symptom(FROM Patients, TO Symptoms)WITH REVERSE_EDGE="reverse_has_symptoms"
    CREATE DIRECTED EDGE symptom_shows_on_disease(FROM Symptoms, TO Disease)WITH REVERSE_EDGE="reverse_shows_on_disease"
    CREATE DIRECTED EDGE disease_refer_to_specialist(FROM Disease, TO Specialist)WITH REVERSE_EDGE="reverse_refers_to_specialist"
''')

print(results)

Successfully created edge types: [patient_shows_on_symptom].
Successfully created reverse edge types: [reverse_has_symptoms].
Successfully created edge types: [symptom_shows_on_disease].
Successfully created reverse edge types: [reverse_shows_on_disease].
Successfully created edge types: [disease_refer_to_specialist].
Successfully created reverse edge types: [reverse_refers_to_specialist].


In [7]:
# Create the Graph Schema

results = conn.gsql('Create GRAPH Umbrella_AI(Patients, Symptoms, Disease, Specialist, patient_shows_on_symptom, symptom_shows_on_disease, disease_refer_to_specialist)')

In [8]:
results = conn.getSchema()
print(json.dumps(results, indent=2))

{
  "GraphName": "UmbrellaAI2",
  "VertexTypes": [],
  "EdgeTypes": [],
  "UDTs": []
}


In [9]:
# SETUP TIGERGRAPH CONNECTION

# graph = tg.TigerGraphConnection(host=hostName, graphname = graphName)
#authToken = graph.getToken(secret)
#authToken = authToken[0]
#print(f"This is the secret token:\n {authToken}")

In [10]:
print(authToken)

raenqapuejlskkn8b81932cq6bjs4qdk


In [11]:
#Import the files 
patient = pd.read_csv("~/Downloads/Umbrella AI/Practice Files/patient_vertex.csv")
symptoms = pd.read_csv("~/Downloads/Umbrella AI/Practice Files/symptom_vertex.csv", sep=",")
diseases = pd.read_csv("~/Downloads/Umbrella AI/Practice Files/Disease_vertex.csv", sep=",")
specialist = pd.read_csv("~/Downloads/Umbrella AI/Practice Files/specialist_vertex.csv", sep=",")
patient_shows_on_symptom = pd.read_csv("~/Downloads/Umbrella AI/Practice Files/patients_symptom_edge.csv", sep=",")
symptom_shows_on_disease = pd.read_csv("~/Downloads/Umbrella AI/Practice Files/symptom_disease_edge.csv", sep=",")
disease_refer_to_specialist = pd.read_csv("~/Downloads/Umbrella AI/Practice Files/diseases_specialist_edge.csv", sep=",")

In [23]:
patient.head()

,id,first_name,last_name,address,age,email,gender,phone
0,1.0,Leia,Norris,6836 Johnston Ave Reevesville South Carolina(S...,29.0,leia.norris@hotmail.com,female,(843) 563-9685
1,2.0,Sadie,Johnson,151 Hilltop Rd Goldonna Louisiana(LA) 71031,43.0,sadie.johnson@gmail.com,female,(318) 727-4959
2,3.0,Jose,Newman,10562 Arleta Ave Mission Hills California(CA) ...,56.0,jose.newman@hotmail.com,male,(818) 897-3613
3,4.0,Liberty,Hart,884 Davis St #303 San Leandro California(CA) 9...,73.0,liberty96@hotmail.com,female,(510) 568-5566
4,5.0,Bella,Wong,2705 Big Buck Dr Lewiston Michigan(MI) 49756,28.0,bella.wong@gmail.com,female,(989) 786-4312


In [24]:
symptoms.head()

,id,symptom
0,1,pain with touching the affected area
1,2,redo tasks because they aren't perfect the fir...
2,3,eating until you're uncomfortably full
3,4,"a dirty white, greenish or black nipple discha..."
4,5,losing weight without trying


In [25]:
diseases.head()

,id,disease
0,1,hypertensive disease
1,2,diabetes
2,3,depression mental
3,4,depressive disorder
4,5,coronary arteriosclerosis


In [26]:
specialist.head()

,id,specialist
0,1,Nutritionist
1,2,Neuropathologist
2,3,ENT specialist
3,4,PM&R Physician
4,5,Radiation Oncology Medical Physicist


In [39]:
symptom_shows_on_disease.head()

,disease_id,disease_name,symptom_id,symptom_name
0,1,['hypertensive disease'],951,['pain chest']
1,1,['hypertensive disease'],880,['shortness of breath']
2,1,['hypertensive disease'],4583,['dizziness']
3,1,['hypertensive disease'],4668,['asthenia']
4,1,['hypertensive disease'],4260,['fall']


# Creating a loading job for all Vertex

In [15]:
results = conn.gsql('''
    USE GRAPH UmbrellaAI2
    CREATE LOADING JOB load_vertex FOR GRAPH UmbrellaAI2 {
    DEFINE FILENAME PatientDataSource;
    DEFINE FILENAME SymptomsDataSource;
    DEFINE FILENAME DiseasesDataSource;
    DEFINE FILENAME SpecialistsDataSource;
    LOAD PatientDataSource TO VERTEX Patients VALUES ($0, $1, $2, $3, $4, $5, $6, $7) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
    LOAD SymptomsDataSource TO VERTEX Symptoms VALUES ($0, $1) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
    LOAD DiseasesDataSource TO VERTEX Disease VALUES ($0, $1) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
    LOAD SpecialistsDataSource TO VERTEX Specialist VALUES ($0, $1) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
    }
''')
print(results)

Using graph 'UmbrellaAI2'
Successfully created loading jobs: [load_vertex].


In [37]:
# Loading the patient data file with the 'load_patients' job
patients_file = '/Users/mac/Downloads/Umbrella AI/Practice Files/patient_vertex.csv'
load_patients = conn.uploadFile(patients_file, fileTag = 'PatientDataSource', jobName = 'load_vertex')
print(json.dumps(load_patients, indent = 2))

[
  {
    "sourceFileName": "Online_POST",
    "statistics": {
      "sourceFileName": "Online_POST",
      "parsingStatistics": {
        "fileLevel": {
          "validLine": 221
        },
        "objectLevel": {
          "vertex": [
            {
              "typeName": "Patients",
              "validObject": 201,
              "noIdFound": {
                "count": 20,
                "sample": {
                  "line": 202,
                  "lineData": ",,,,,,,\r\n"
                }
              }
            }
          ],
          "edge": [],
          "deleteVertex": [],
          "deleteEdge": []
        }
      }
    }
  }
]


In [38]:
# Loading the symptoms data file with the 'load_symptoms' job
symptoms_file = '/Users/mac/Downloads/Umbrella AI/Practice Files/symptom_vertex.csv'
symptoms_loading = conn.runLoadingJobWithFile(symptoms_file,fileTag = 'SymptomsDataSource', jobName = 'load_vertex')
print(json.dumps(symptoms_loading, indent = 2))

[
  {
    "sourceFileName": "Online_POST",
    "statistics": {
      "sourceFileName": "Online_POST",
      "parsingStatistics": {
        "fileLevel": {
          "validLine": 5981
        },
        "objectLevel": {
          "vertex": [
            {
              "typeName": "Symptoms",
              "validObject": 5981
            }
          ],
          "edge": [],
          "deleteVertex": [],
          "deleteEdge": []
        }
      }
    }
  }
]


In [39]:
# Loading the diseases data file with the 'load_diseases' job
diseases_file = '/Users/mac/Downloads/Umbrella AI/Practice Files/Disease_vertex.csv'
results = conn.uploadFile(diseases_file, fileTag = 'DiseasesDataSource', jobName = 'load_vertex')
print(json.dumps(results, indent = 2))

[
  {
    "sourceFileName": "Online_POST",
    "statistics": {
      "sourceFileName": "Online_POST",
      "parsingStatistics": {
        "fileLevel": {
          "validLine": 1674
        },
        "objectLevel": {
          "vertex": [
            {
              "typeName": "Disease",
              "validObject": 1674
            }
          ],
          "edge": [],
          "deleteVertex": [],
          "deleteEdge": []
        }
      }
    }
  }
]


In [40]:
#Loading the specialists data file with the 'load_specialists' job

specialist_file = '/Users/mac/Downloads/Umbrella AI/Practice Files/specialist_vertex.csv'
results = conn.uploadFile(specialist_file, fileTag = 'SpecialistsDataSource', jobName = 'load_vertex')
print(json.dumps(results, indent = 2))

[
  {
    "sourceFileName": "Online_POST",
    "statistics": {
      "sourceFileName": "Online_POST",
      "parsingStatistics": {
        "fileLevel": {
          "validLine": 170
        },
        "objectLevel": {
          "vertex": [
            {
              "typeName": "Specialist",
              "validObject": 170
            }
          ],
          "edge": [],
          "deleteVertex": [],
          "deleteEdge": []
        }
      }
    }
  }
]


# Creating a loading job for Diseases - all Edge

In [20]:
results = conn.gsql('''
    USE GRAPH UmbrellaAI2
    CREATE LOADING JOB load_edge FOR GRAPH UmbrellaAI2 {
    DEFINE FILENAME Patient_to_SymptomsDataSource;
    DEFINE FILENAME Symptoms_to_DiseasesDataSource;
    DEFINE FILENAME Diseases_to_SpecialistsDataSource;
    LOAD Patient_to_SymptomsDataSource TO EDGE patient_shows_on_symptom VALUES ($0, $1) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
    LOAD Symptoms_to_DiseasesDataSource TO EDGE symptom_shows_on_disease VALUES ($2, $0) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
    LOAD Diseases_to_SpecialistsDataSource TO EDGE disease_refer_to_specialist VALUES ($0, $2) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
    }
''')
print(results)

Using graph 'UmbrellaAI2'
Successfully created loading jobs: [load_edge].


In [41]:
# Loading the patients_to_symptoms data file with the 'load_patient_shows_on_symptom' job
patient_shows_on_symptom_file = '/Users/mac/Downloads/Umbrella AI/Practice Files/patients_symptom_edge.csv'
results = conn.uploadFile(patient_shows_on_symptom_file, fileTag = 'Patient_to_SymptomsDataSource', jobName = 'load_edge')
print(json.dumps(results, indent = 2))

[
  {
    "sourceFileName": "Online_POST",
    "statistics": {
      "sourceFileName": "Online_POST",
      "parsingStatistics": {
        "fileLevel": {
          "validLine": 18658
        },
        "objectLevel": {
          "vertex": [],
          "edge": [
            {
              "typeName": "patient_shows_on_symptom",
              "validObject": 18658
            }
          ],
          "deleteVertex": [],
          "deleteEdge": []
        }
      }
    }
  }
]


In [42]:
# Loading the symptom_shows_on_disease data file with the 'load_patient_shows_on_symptom' job

symptom_shows_on_disease_file = '/Users/mac/Downloads/Umbrella AI/Practice Files/symptom_disease_edge.csv'
results = conn.uploadFile(symptom_shows_on_disease_file, fileTag = 'Symptoms_to_DiseasesDataSource', jobName = 'load_edge')
print(json.dumps(results, indent = 2))

[
  {
    "sourceFileName": "Online_POST",
    "statistics": {
      "sourceFileName": "Online_POST",
      "parsingStatistics": {
        "fileLevel": {
          "validLine": 18658
        },
        "objectLevel": {
          "vertex": [],
          "edge": [
            {
              "typeName": "symptom_shows_on_disease",
              "validObject": 18658
            }
          ],
          "deleteVertex": [],
          "deleteEdge": []
        }
      }
    }
  }
]


In [43]:
# Loading the disease_refer_to_specialist data file with the 'load_patient_shows_on_symptom' job

disease_refer_to_specialist_file = '/Users/mac/Downloads/Umbrella AI/Practice Files/diseases_specialist_edge.csv'
results = conn.uploadFile(disease_refer_to_specialist_file, fileTag = 'Diseases_to_SpecialistsDataSource', jobName = 'load_edge')
print(json.dumps(results, indent = 2))

[
  {
    "sourceFileName": "Online_POST",
    "statistics": {
      "sourceFileName": "Online_POST",
      "parsingStatistics": {
        "fileLevel": {
          "validLine": 3750
        },
        "objectLevel": {
          "vertex": [],
          "edge": [
            {
              "typeName": "disease_refer_to_specialist",
              "validObject": 3750
            }
          ],
          "deleteVertex": [],
          "deleteEdge": []
        }
      }
    }
  }
]


In [44]:
# Exploring thee Graph

# Get Vertex and Edge Schema


results = conn.getVertexTypes()
print(f"Verticiees: {results}")
vertices = results

results = conn.getEdgeTypes()
print(f"Edges: {results}")
edges = results

Verticiees: []
Edges: []


In [35]:
print("Vertex Counts")
for vertex in vertices:
    print(f"There are {conn.getVertexCount(vertex)} {vertex} vertices in the graph")
    
print("---------------")
print("Edge Counts")
for edge in edges:
    print(f"There are {conn.getEdgeCount(edge)} {edge} vertices in the graph")

Vertex Counts
---------------
Edge Counts


In [64]:
len(patient)

220

In [61]:
len(symptoms)

5980

In [66]:
len(diseases)

1673

In [67]:
len(specialist)

169

In [10]:
len(patient_shows_on_symptom)

18657

In [11]:
len(symptom_shows_on_disease)

18657

In [12]:
len(disease_refer_to_specialist)

3749

# TabPy: Connect with Tableau

In [5]:
from notebook import notebookapp
port_list = [note["port"] for note in notebookapp.list_running_servers()]
port_list

[8888, 8888]

In [12]:
pip install virtualenv 

Note: you may need to restart the kernel to use updated packages.


In [2]:
import virtualenv

In [45]:
def encode(input):     
  le = preproces2ing.LabelEncoder()
  return pd.DataFrame({
    'Opportunity Number' : input['Opportunity Number'],
    'Supplies Subgroup Encoded' : le.fit_transform(input['Supplies Subgroup']),
    'Region Encoded' : le.fit_transform(input['Region']),
    'Route To Market Encoded' : le.fit_transform(input['Route To Market']),
    'Opportunity Result Encoded' : le.fit_transform(input['Opportunity Result']),
    'Competitor Type Encoded' : le.fit_transform(input['Competitor Type']),
    'Supplies Group Encoded' : le.fit_transform(input['Supplies Group']),
})